In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [2]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [143]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(6244, 313)

In [144]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [145]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [146]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('KO_YU_JIN', '고유진', '블록베리 크리에이티브')

In [147]:
%time KO_YU_JIN = make_produce48_df(name)

  0%|          | 0/313 [00:00<?, ?it/s]

고유진의 댓글 수: 6244
이전 댓글 모두 불러오기 시작...


  0%|          | 0/6244 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 6244/6244 [1:31:16<00:00,  1.14it/s]

making dataframe done!
CPU times: user 50.5 s, sys: 4.2 s, total: 54.7 s
Wall time: 1h 51min 42s


In [148]:
KO_YU_JIN.shape

(6244, 3)

In [149]:
KO_YU_JIN.tail()

,User_ID,Comment,Date
6239,Hanan,♡♡♡,2018-05-11
6240,조유진,유진아 데뷔하자!!!!,2018-05-11
6241,조유진,유진아ㅜㅜㅠㅠㅠㅠㅠ 언니가 많이 사랑해!!!!! 데뷔하자!!!!!!,2018-05-11
6242,keemhans.,아름다운,2018-05-11
6243,최원정,good~~,2018-05-11


In [150]:
pickle.dump(KO_YU_JIN, open('./produce48/블록베리 크리에이티브/KO_YU_JIN.pkl','wb'))

In [151]:
KO_YU_JIN_df = pickle.load(open('./produce48/블록베리 크리에이티브/KO_YU_JIN.pkl','rb'))

In [152]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [153]:
KO_YU_JIN_df.tail()

,User_ID,Comment,Date
6239,Hanan,♡♡♡,2018-05-11
6240,조유진,유진아 데뷔하자!!!!,2018-05-11
6241,조유진,유진아ㅜㅜㅠㅠㅠㅠㅠ 언니가 많이 사랑해!!!!! 데뷔하자!!!!!!,2018-05-11
6242,keemhans.,아름다운,2018-05-11
6243,최원정,good~~,2018-05-11


In [154]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기